### Инициализация Модели

In [1]:
from transformers import AutoTokenizer,AutoModelForCausalLM
import torch

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')



# Загрузка и инициализация модели и токенайзера
model_name = "sberbank-ai/rugpt3medium_based_on_gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name, bos_token="[START]", eos_token ="[END]", pad_token = '<pad>')
model = AutoModelForCausalLM.from_pretrained(model_name).to(DEVICE)
model.resize_token_embeddings(len(tokenizer))


c:\Users\ugl\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Embedding(50260, 1024)

In [2]:
import torch
torch.cuda.is_available()

True

### Загрузка Датасета и Препроцессинг

In [5]:
from transformers import DataCollatorForLanguageModeling
from datasets import Dataset


train_path = 'train_dataset.txt'
with open(train_path, "w",encoding='utf-8') as f,open('result.txt',encoding='utf-8') as w:
    ptext = w.read()
    f.write(ptext)

# Преобразование в датасет и токенизация
train_dataset = Dataset.from_text(train_path)
train_dataset = train_dataset.map(lambda examples: tokenizer(examples["text"]),remove_columns=['text'])
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])

# Создание даталодера (нарезает текст на оптимальные по длине куски)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

Using custom data configuration default-eb9463226a959ee7


Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 501.83it/s]


Dataset text downloaded and prepared to C:/Users/ugl/.cache/huggingface/datasets/text/default-eb9463226a959ee7/0.0.0. Subsequent calls will reuse this data.


100%|██████████| 9971/9971 [00:09<00:00, 1051.48ex/s]


In [7]:
print(train_dataset[800])
print([tokenizer.decode(x) for x in train_dataset[8000]['input_ids']])

{'input_ids': tensor([50257,  9676,   298, 27316,    30,  5786,  8046,  5764,   927,   281,
         5578,    16,   437,  1514,  2156,  1276,   927,  3149,    18,   889,
          939,  8826,   282, 14469,    30,  7006,   520,  1085,    17,   299,
        32265, 47607,   309,  6045, 36106,   670, 17970, 14032,   329,  1304,
          309,  3427,    18,  1012,  7936, 47345, 11858,   281, 10836,    16,
          281,  5045,   898,   282,   281,  3008,   298,    16,   827, 43184,
           16,   670, 14202, 11273,  2630,    18, 12105,   923,   318, 14533,
        32142,   427,   309,  3705,   289,  1642,    16,  3888,  1101,   360,
          724,   467,  1143,  1612,    16,   828,   387, 23653,    18,   740,
          477,  1514,   942, 12599,    16,   827, 21581, 14472,   289,  9795,
         9351,    18, 17580, 13004,  2299,  9378,   541, 33609,    16,   282,
          970,  2875,  3498, 30931,  5856,    18, 50258]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

### Подготовка для файнтюнинг

In [4]:
#!g1.1
from transformers import Trainer, TrainingArguments


training_args = TrainingArguments(
    output_dir="DescriptionGenerateModel",
    overwrite_output_dir=True,
    num_train_epochs=45, # число эпох
    per_device_train_batch_size=8, # число обучающих примеров в одной итерации
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=8, # накопление градиента
    warmup_steps=10, # "разогревающие шаги"
    save_steps = 7000, # образование папки чекпоинта после этого числа шагов
)

# Класс Trainer предоставляет API для полнофункционального обучения работе с PyTorch для большинства стандартных случаев использования
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset, 
    optimizers = (torch.optim.AdamW(model.parameters(),lr=1e-5),None)
)

### Дообучение

In [7]:
trainer.train()

***** Running training *****
  Num examples = 10026
  Num Epochs = 10
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 50130
  Number of trainable parameters = 125228544
  0%|          | 2/50130 [00:00<2:29:42,  5.58it/s]

KeyboardInterrupt: 